In [ ]:
import os
import shutil
import itertools
import pathlib
from PIL import Image
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2
#Data handling tools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report

#Deep learning libs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Flatten , Activation , Dense , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adadelta, Adagrad, Adam, Adamax, Nadam, SGD, RMSprop

#Warningds
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data_path = "F:\\Mahi\\brain mri\\kaggaleblurred\\train"

filepaths =[]

labels = []

folds = os.listdir(train_data_path)

for fold in folds:
    f_path = os.path.join(train_data_path , fold)
    filelists = os.listdir(f_path)

    for file in filelists:
        filepaths.append(os.path.join(f_path , file))
        labels.append(fold)

#Concat data paths with labels
Fseries = pd.Series(filepaths , name = 'filepaths')
Lseries = pd.Series(labels , name = 'label')
train_df = pd.concat([Fseries , Lseries] , axis = 1)

In [ ]:
test_data_path = "F:\\Mahi\\brain mri\\kaggaleblurred\\test"


filepaths =[]
labels = []

folds = os.listdir(test_data_path)

for fold in folds:
    f_path = os.path.join(test_data_path , fold)
    filelists = os.listdir(f_path)

    for file in filelists:
        filepaths.append(os.path.join(f_path , file))
        labels.append(fold)

#Concat data paths with labels
Fseries = pd.Series(filepaths , name = 'filepaths')
Lseries = pd.Series(labels , name = 'label')
test_df = pd.concat([Fseries , Lseries] , axis = 1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_size = (224 ,224)
batch_size = 16

ts_gen= ImageDataGenerator()

tr_gen= ImageDataGenerator()
# Create an instance of the ImageDataGenerator

train_gen = tr_gen.flow_from_dataframe(train_df , x_col = 'filepaths' , y_col = 'label' , target_size = img_size ,
                                      class_mode = 'categorical' , color_mode = 'rgb' , shuffle = True , batch_size =batch_size)
test_gen = ts_gen.flow_from_dataframe(test_df , x_col= 'filepaths' , y_col = 'label' , target_size = img_size, class_mode = 'categorical' ,
                                      color_mode= 'rgb', shuffle = False , batch_size = batch_size)

In [ ]:
gen_dict = train_gen.class_indices
classes = list(gen_dict.keys())
images , labels = next(train_gen)

plt.figure(figsize= (20,20))

for i in range(16):
    plt.subplot(4,4,i+1)
    image = images[i] / 255
    plt.imshow(image)
    index = np.argmax(labels[i])
    class_name = classes[index]
    plt.title(class_name , color = 'blue' , fontsize= 12)
    plt.axis('off')
plt.show();

In [ ]:
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Specify the path to save the best model weights
checkpoint_filepath = 'F:\\Mahi\\brain mri\\cnn7022.h5'
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
# Set up the ModelCheckpoint callback to save weights with the highest validation accuracy
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
#Adam

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()# contrast change by up to 50%)

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))

#
Adamax

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
   # epochs=15,
    #validation_data=test_generator,
    #validation_steps=69,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))

Adadelta

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=Adadelta(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
   # epochs=15,
    #validation_data=test_generator,
    #validation_steps=69,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))

Adagrad

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()# contrast change by up to 50%)

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=Adagrad(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
   # epochs=15,
    #validation_data=test_generator,
    #validation_steps=69,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))

Nadam

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()# contrast change by up to 50%)

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=Nadam(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
   # epochs=15,
    #validation_data=test_generator,
    #validation_steps=69,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))

SGD

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()# contrast change by up to 50%)

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
   # epochs=15,
    #validation_data=test_generator,
    #validation_steps=69,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))

RMSprop

In [ ]:
#Best so far
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers.experimental import preprocessing

model2 = Sequential()# contrast change by up to 50%)

# Convolutional layer 1
model2.add(Conv2D(64, (4, 4), activation="relu", input_shape=(224, 224, 3), padding='same'))
model2.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model2.add(BatchNormalization())
#conv2
model2.add(Conv2D(64,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

#Convolutional layer 3
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 4
model2.add(Conv2D(128,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

# Convolutional layer 5
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

 # Convolutional layer 6
model2.add(Conv2D(256,(4,4), padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))


model2.add(GlobalAveragePooling2D())

# Full connect layers

model2.add(Dense(units= 1024, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=512, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(units=4, activation='softmax'))



model2.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy',
                   metrics=['accuracy'])
history = model2.fit(
    x=train_gen,
    epochs = 100,
    validation_data = test_gen,
    steps_per_epoch=119,
   # epochs=15,
    #validation_data=test_generator,
    #validation_steps=69,
    callbacks=[reduce_lr, model_checkpoint_callback])
#keras.utils.plot_model(model2, to_file='CNN 95.56%_model_architecture.png',show_shapes=True,)

In [ ]:

preds = model2.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(y_pred,test_gen.classes,digits=6))